## Linear Regression

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:
df = pd.read_csv('/Users/calebreed/Documents/GitHub/DATA-4950-Capstone/DATA-4950-Capstone/data/external/salaries_clean.csv')
# drop the outliers
df = df[df.annual_base_pay < 1000000]
# drop unneccessary variables
df = df.drop(["index","salary_id","location_latitude","location_longitude","comments","submitted_at"],axis=1)
# job_title_rank, location_state, and location_country have too many missing values so they will be dropped
df = df.drop(['job_title_rank','location_state','location_country'],axis=1)
# drops employer experience years
df = df.drop("employer_experience_years",axis=1)
# drops columns with too many unique values
df = df.drop("employer_name",axis=1)
df = df.drop("job_title",axis=1)
df = df.drop("location_name",axis=1) 
# fill experience missing values with the mean
df.loc[:, "total_experience_years"] = df.loc[:, "total_experience_years"].fillna(df["total_experience_years"].mean())
df.loc[:, "annual_base_pay"] = df.loc[:, "annual_base_pay"].fillna(df["annual_base_pay"].mean())
df.loc[:, "signing_bonus"] = df.loc[:, "signing_bonus"].fillna(method='ffill')
df.loc[:, "annual_bonus"] = df.loc[:, "annual_bonus"].fillna(method='ffill')
# stock bonus is a string so will be dropping it
df = df.drop("stock_value_bonus",axis=1)
df.head()

,job_title_category,total_experience_years,annual_base_pay,signing_bonus,annual_bonus
0,Engineering,13.0,125000.0,5000.0,0.0
1,Software,15.0,65000.0,5000.0,5000.0
2,Software,4.0,86000.0,5000.0,6000.0
3,Other,4.0,105000.0,5000.0,8500.0
4,Software,4.0,110000.0,5000.0,7000.0


In [3]:
# creates dummy variables for job category
dummies = pd.get_dummies(df['job_title_category'], prefix='job_category')
dummies.head()

,job_category_Applied Science,job_category_Data,job_category_Engineering,job_category_Management,job_category_Operations,job_category_Other,job_category_Software,job_category_Web
0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0


In [4]:
# merges dummy variables with dataframe and drops original column
df = pd.concat([df, dummies], axis=1)
df = df.drop('job_title_category', axis=1)
df.head()

,total_experience_years,annual_base_pay,signing_bonus,annual_bonus,job_category_Applied Science,job_category_Data,job_category_Engineering,job_category_Management,job_category_Operations,job_category_Other,job_category_Software,job_category_Web
0,13.0,125000.0,5000.0,0.0,0,0,1,0,0,0,0,0
1,15.0,65000.0,5000.0,5000.0,0,0,0,0,0,0,1,0
2,4.0,86000.0,5000.0,6000.0,0,0,0,0,0,0,1,0
3,4.0,105000.0,5000.0,8500.0,0,0,0,0,0,1,0,0
4,4.0,110000.0,5000.0,7000.0,0,0,0,0,0,0,1,0


In [5]:
# seperates features into x and y variables
X = df.drop('annual_base_pay', axis = 1)

y = df['annual_base_pay'] 

# splits data into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [6]:
# builds intial model with all predictors
X_reg = X_train
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     16.68
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           2.01e-28
Time:                        20:29:14   Log-Likelihood:                -14529.
No. Observations:                1141   AIC:                         2.908e+04
Df Residuals:                    1130   BIC:                         2.914e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         6.186e+04   7053.403      8.770      0.000     4.8e+04    7.57e+04
total_experience_years        3187.0666    467.654      6.815      0.000    2269.499    4104.634
signing_bonus                    0.2957      0.096      3.094      0.002       0.108       0.483
annual_bonus                     0.7041      0.086      8.210      0.000       0.536       0.872
job_category_Applied Science  5660.4905   3.03e+04      0.187      0.852   -5.39e+04    6.52e+04
job_category_Data             1387.5882   1.17e+04      0.119      0.906   -2.16e+04    2.43e+04
job_category_Engineering      8752.2058   9120.462      0.960      0.337   -9142.739    2.66e+04
job_category_Management       2.461e+04   9978.691      2.466      0.014    5026.400    4.42e+04
job_category_Operations      -9154.2905   3.69e+04     -0.248      0.804   -8.16e+04    6.33e+04
job_category_Other            1.928e+04   8256.766      2.335      0.020    3077.243    3.55e+04
job_category_Software          1.49e+04   7004.409      2.128      0.034    1160.876    2.86e+04
job_category_Web             -3576.9605   1.11e+04     -0.323      0.747   -2.53e+04    1.82e+04
==============================================================================
Omnibus:                     1156.864   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68199.311
Skew:                           4.769   Prob(JB):                         0.00
Kurtosis:                      39.654   Cond. No.                     4.55e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.13e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [7]:
X_reg = X_reg.drop(['job_category_Data'],axis=1)
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     16.68
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           2.01e-28
Time:                        20:29:14   Log-Likelihood:                -14529.
No. Observations:                1141   AIC:                         2.908e+04
Df Residuals:                    1130   BIC:                         2.914e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                         6.324e+04   1.13e+04      5.583      0.000     4.1e+04    8.55e+04
total_experience_years        3187.0666    467.654      6.815      0.000    2269.499    4104.634
signing_bonus                    0.2957      0.096      3.094      0.002       0.108       0.483
annual_bonus                     0.7041      0.086      8.210      0.000       0.536       0.872
job_category_Applied Science  4272.9023   3.55e+04      0.121      0.904   -6.53e+04    7.38e+04
job_category_Engineering      7364.6176   1.34e+04      0.548      0.584    -1.9e+04    3.38e+04
job_category_Management       2.322e+04   1.42e+04      1.629      0.103   -4738.706    5.12e+04
job_category_Operations      -1.054e+04   4.28e+04     -0.246      0.805   -9.45e+04    7.34e+04
job_category_Other            1.789e+04   1.27e+04      1.406      0.160   -7079.768    4.29e+04
job_category_Software         1.352e+04   1.16e+04      1.161      0.246   -9319.542    3.64e+04
job_category_Web             -4964.5487   1.52e+04     -0.327      0.744   -3.47e+04    2.48e+04
==============================================================================
Omnibus:                     1156.864   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68199.311
Skew:                           4.769   Prob(JB):                         0.00
Kurtosis:                      39.654   Cond. No.                     6.08e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.08e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
X_reg = X_reg.drop(['job_category_Applied Science'],axis=1)
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     18.55
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           4.55e-29
Time:                        20:29:14   Log-Likelihood:                -14529.
No. Observations:                1141   AIC:                         2.908e+04
Df Residuals:                    1131   BIC:                         2.913e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     6.366e+04   1.08e+04      5.897      0.000    4.25e+04    8.48e+04
total_experience_years    3188.7455    467.243      6.825      0.000    2271.985    4105.506
signing_bonus                0.2956      0.096      3.095      0.002       0.108       0.483
annual_bonus                 0.7041      0.086      8.214      0.000       0.536       0.872
job_category_Engineering  6939.9276    1.3e+04      0.535      0.593   -1.85e+04    3.24e+04
job_category_Management   2.279e+04   1.38e+04      1.652      0.099   -4274.061    4.99e+04
job_category_Operations  -1.097e+04   4.26e+04     -0.257      0.797   -9.46e+04    7.26e+04
job_category_Other        1.746e+04   1.22e+04      1.429      0.153   -6513.811    4.14e+04
job_category_Software     1.309e+04   1.11e+04      1.180      0.238   -8675.920    3.49e+04
job_category_Web         -5387.1504   1.48e+04     -0.365      0.715   -3.43e+04    2.36e+04
==============================================================================
Omnibus:                     1156.838   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68194.747
Skew:                           4.769   Prob(JB):                         0.00
Kurtosis:                      39.653   Cond. No.                     5.94e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.94e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
X_reg = X_reg.drop(['job_category_Operations'],axis=1)
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     20.88
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           9.90e-30
Time:                        20:29:14   Log-Likelihood:                -14529.
No. Observations:                1141   AIC:                         2.908e+04
Df Residuals:                    1132   BIC:                         2.912e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     6.302e+04   1.05e+04      6.000      0.000    4.24e+04    8.36e+04
total_experience_years    3180.8831    466.051      6.825      0.000    2266.461    4095.305
signing_bonus                0.2956      0.095      3.096      0.002       0.108       0.483
annual_bonus                 0.7041      0.086      8.218      0.000       0.536       0.872
job_category_Engineering  7633.9445   1.27e+04      0.602      0.547   -1.73e+04    3.25e+04
job_category_Management    2.35e+04   1.35e+04      1.739      0.082   -3012.732       5e+04
job_category_Other        1.816e+04   1.19e+04      1.525      0.128   -5208.293    4.15e+04
job_category_Software     1.378e+04   1.08e+04      1.279      0.201   -7360.457    3.49e+04
job_category_Web         -4704.7036   1.45e+04     -0.324      0.746   -3.32e+04    2.38e+04
==============================================================================
Omnibus:                     1156.773   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68180.642
Skew:                           4.768   Prob(JB):                         0.00
Kurtosis:                      39.649   Cond. No.                     3.56e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.56e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
X_reg = X_reg.drop(['job_category_Web'],axis=1)
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     23.87
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           2.04e-30
Time:                        20:29:14   Log-Likelihood:                -14529.
No. Observations:                1141   AIC:                         2.907e+04
Df Residuals:                    1133   BIC:                         2.912e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     6.071e+04   7712.432      7.872      0.000    4.56e+04    7.58e+04
total_experience_years    3176.4867    465.670      6.821      0.000    2262.814    4090.159
signing_bonus                0.2959      0.095      3.101      0.002       0.109       0.483
annual_bonus                 0.7043      0.086      8.223      0.000       0.536       0.872
job_category_Engineering  9974.8636   1.04e+04      0.957      0.339   -1.05e+04    3.04e+04
job_category_Management   2.584e+04   1.14e+04      2.266      0.024    3466.257    4.82e+04
job_category_Other        2.051e+04   9463.915      2.167      0.030    1937.092    3.91e+04
job_category_Software     1.611e+04   8015.412      2.010      0.045     380.693    3.18e+04
==============================================================================
Omnibus:                     1156.186   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            68085.548
Skew:                           4.765   Prob(JB):                         0.00
Kurtosis:                      39.624   Cond. No.                     2.38e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.38e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
X_reg = X_reg.drop(['job_category_Engineering'],axis=1)
X_reg = sm.add_constant(X_reg) 
 
reg1 = sm.OLS(y_train, X_reg).fit()
pred = reg1.predict(X_reg) 
 
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     27.69
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           5.77e-31
Time:                        20:29:14   Log-Likelihood:                -14530.
No. Observations:                1141   AIC:                         2.907e+04
Df Residuals:                    1134   BIC:                         2.911e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     6.53e+04   6038.637     10.814      0.000    5.35e+04    7.71e+04
total_experience_years   3215.5596    463.859      6.932      0.000    2305.441    4125.678
signing_bonus               0.2991      0.095      3.136      0.002       0.112       0.486
annual_bonus                0.7054      0.086      8.237      0.000       0.537       0.873
job_category_Management  2.084e+04   1.01e+04      2.056      0.040     954.146    4.07e+04
job_category_Other       1.557e+04   7934.664      1.962      0.050       2.208    3.11e+04
job_category_Software    1.125e+04   6201.411      1.814      0.070    -919.248    2.34e+04
==============================================================================
Omnibus:                     1154.235   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            67711.772
Skew:                           4.752   Prob(JB):                         0.00
Kurtosis:                      39.523   Cond. No.                     1.66e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
import numpy as np
# Create a list of statistically significant variables to use for the final model
results = reg1.params.reset_index()
results = pd.DataFrame(results)
stat_sig_Pred = results['index'].values
display(stat_sig_Pred)
stat_sig_Pred = np.delete(stat_sig_Pred, 0)
stat_sig_Pred

array(['const', 'total_experience_years', 'signing_bonus', 'annual_bonus',
       'job_category_Management', 'job_category_Other',
       'job_category_Software'], dtype=object)

array(['total_experience_years', 'signing_bonus', 'annual_bonus',
       'job_category_Management', 'job_category_Other',
       'job_category_Software'], dtype=object)

In [18]:
# Final model
X_train_new = X_reg[stat_sig_Pred]
X_train_new = sm.add_constant(X_train_new)

reg = sm.OLS(y_train, X_train_new).fit()
pred2 = reg.predict(X_train_new) 

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_base_pay   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                     27.69
Date:                Mon, 24 Apr 2023   Prob (F-statistic):           5.77e-31
Time:                        20:38:35   Log-Likelihood:                -14530.
No. Observations:                1141   AIC:                         2.907e+04
Df Residuals:                    1134   BIC:                         2.911e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     6.53e+04   6038.637     10.814      0.000    5.35e+04    7.71e+04
total_experience_years   3215.5596    463.859      6.932      0.000    2305.441    4125.678
signing_bonus               0.2991      0.095      3.136      0.002       0.112       0.486
annual_bonus                0.7054      0.086      8.237      0.000       0.537       0.873
job_category_Management  2.084e+04   1.01e+04      2.056      0.040     954.146    4.07e+04
job_category_Other       1.557e+04   7934.664      1.962      0.050       2.208    3.11e+04
job_category_Software    1.125e+04   6201.411      1.814      0.070    -919.248    2.34e+04
==============================================================================
Omnibus:                     1154.235   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            67711.772
Skew:                           4.752   Prob(JB):                         0.00
Kurtosis:                      39.523   Cond. No.                     1.66e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.66e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
train_mse = sm.tools.eval_measures.mse(y_train, pred2, axis=0)
train_rmse= sm.tools.eval_measures.rmse(y_train, pred2, axis=0)
train_mae= sm.tools.eval_measures.meanabs(y_train, pred2, axis=0)
train_rmspe= sm.tools.eval_measures.rmspe(y_train, pred2, axis=0)

print(f' the train MSE is:{train_mse}')
print(f' the train RMSE is:{train_rmse}')
print(f' the train MAE is:{train_mae}')
print(f' the train RMSPE is:{train_rmspe}')

 the train MSE is:6735822877.627717
 the train RMSE is:82072.05905561111
 the train MAE is:43551.726434584656
 the train RMSPE is:2447.8263029924847


In [20]:
# Predict on the test data
X_test_new = X_test[stat_sig_Pred] 
X_test_new = sm.add_constant(X_test_new)


# Calculate the estimated y values using the test dataset
y_hat_test = reg.predict(X_test_new)

In [22]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
test_mse = mean_squared_error(y_test, y_hat_test)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_hat_test)
test_r_squared = r2_score(y_test, y_hat_test)
test_mape = mean_absolute_percentage_error(y_test, y_hat_test)

print(f'The test RMSE is:\t{round(test_rmse, 2)}')
print(f'The test MAE is:\t{round(test_mae, 2)}')
print(f'The test Rsquared is:\t{round(test_r_squared, 3)}')
print(f'The test MAPE is:\t{round(test_mape, 2)}')

The test RMSE is:	63608.64
The test MAE is:	42236.81
The test Rsquared is:	0.03
The test MAPE is:	9.817220382626676e+17
